In [1]:
import subprocess
import os
from LAMMPS_gen_in_file import * 
from LAMMPS_data_preprocess import * 
from LAMMPS_analysis import *

velocities = [0.1, 0.5, 1, 1.5]
temperature = 300

test_num = 1

            
for velocity in velocities:
    dump_dir = "./runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    dump_dir_ = "runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    
    try:
        os.mkdir("./runs")
    except FileExistsError:
        print('Folder already exist.')
        
    try:
        os.mkdir("./runs/test_" + str(1))
    except FileExistsError:
        print('Folder already exist.')
        
    
    generate_in_file(velocity, temperature, dump_dir=dump_dir, output_file = dump_dir_ + "/in.perforation", 
                     iterations = [1000, 1000, 0, 10000])
    
    #os.system('cp ffield_reax.cho2016 ' + dump_dir_ + '/ffield_reax.cho2016')
    os.system('cp C60.data ' + dump_dir_ + '/C60.data')
    
    prepare_data(graphene_size=[40, 40], input_c60_file = dump_dir_ + '/C60.data', 
                 output_c60_centered = dump_dir_ + '/centered_C60.data', 
                 output_graphene_centered = dump_dir_ + '/centered_graphene.data', 
                 output_file= dump_dir_ + '/combined.data', 
                 output_graphene = dump_dir_ + '/graphene.data', 
                 z_graphene = 20, z_c60 = 5)

    lammps_path = "/home/korotnev/Documents/LAMMPS/lammps-29Aug2024/src/lmp_serial"  

    script_path = dump_dir_ + '/in.perforation'

    try:
        result = subprocess.run([lammps_path, '-in', script_path], check=True, 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("LAMMPS simulation started successfully!")
        print(result.stdout.decode())  # Вывод успешных сообщений
        print(result.stderr.decode())  # Вывод сообщений об ошибках
        
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running LAMMPS: {e}")

In File generated.

Preparing data started.

Graphene structure with 1080 atoms has been written to runs/test_1/vel_0.1_temp_300/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared


KeyboardInterrupt: 

In [1]:
import subprocess
import os
from LAMMPS_gen_in_file import * 
from LAMMPS_data_preprocess import * 
from concurrent.futures import ProcessPoolExecutor

velocities = [0.02, 0.05, 0.1, 0.5]
temperature = 300

test_num = 1

def run_lammps_simulation(velocity, temperature):
    dump_dir = "./runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    dump_dir_ = "runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    
    try:
        os.mkdir("./runs")
    except FileExistsError:
        print('Folder already exist.')
        
    try:
        os.mkdir("./runs/test_" + str(1))
    except FileExistsError:
        print('Folder already exist.')

    generate_in_file(velocity, temperature, dump_dir=dump_dir, output_file=dump_dir_ + "/in.perforation", 
                     iterations=[1000, 1000, 0, 50000], ts = [0.1, 0.1, 0.05])
    
    os.system(f'cp C60.data {dump_dir_}/C60.data')
    
    prepare_data(graphene_size=[20, 20], input_c60_file=f'{dump_dir_}/C60.data', 
                 output_c60_centered=f'{dump_dir_}/centered_C60.data', 
                 output_graphene_centered=f'{dump_dir_}/centered_graphene.data', 
                 output_file=f'{dump_dir_}/combined.data', 
                 output_graphene=f'{dump_dir_}/graphene.data', 
                 z_graphene=20, z_c60=5)

    lammps_path = "/home/korotnev/Documents/LAMMPS/lammps-29Aug2024/src/lmp_serial"  
    script_path = f'{dump_dir_}/in.perforation'

    try:
        result = subprocess.run([lammps_path, '-in', script_path], check=True, 
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"LAMMPS simulation for velocity {velocity} started successfully!")
        print(result.stdout.decode())  # Вывод успешных сообщений
        print(result.stderr.decode())  # Вывод сообщений об ошибках
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running LAMMPS for velocity {velocity}: {e}")
        
    df_graphene = get_dataset(get_files(dump_dir_, el='gr'))
    df_c60 = get_dataset(get_files(dump_dir_, el='c60'))
    
    get_plots_coords_velocities(df_graphene, name=dump_dir_ + '/graphene')
    get_plots_coords_velocities(df_c60, name=dump_dir_ + '/c60')
    
    print(f'Фуллерен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken(df_c60, alpha = 0.03)}')
    print(f'Графен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken(df_graphene, alpha = 0.03)}')

    

# Используем ProcessPoolExecutor для параллельной обработки
with ProcessPoolExecutor() as executor:
    # Запускаем задачу для каждой скорости
    futures = [executor.submit(run_lammps_simulation, velocity, temperature) for velocity in velocities]
    
    # Ожидаем завершения всех задач
    for future in futures:
        future.result()  # Это заблокирует основной поток до завершения задачи

Folder already exist.Folder already exist.Folder already exist.Folder already exist.



Folder already exist.Folder already exist.Folder already exist.Folder already exist.



In File generated.
In File generated.
In File generated.
In File generated.




Preparing data started.
Preparing data started.
Preparing data started.
Preparing data started.




Graphene structure with 270 atoms has been written to runs/test_1/vel_0.05_temp_300/graphene.data.Graphene structure with 270 atoms has been written to runs/test_1/vel_0.02_temp_300/graphene.data.Graphene structure with 270 atoms has been written to runs/test_1/vel_0.5_temp_300/graphene.data.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.1_temp_300/graphene.data.Graphene has generated. Centering data started.
Graphene has generated. Centering data started.




Graphene has generated. Centering data started.
Graphene has generated. Centering data started.


Data centered. Merging data started.
Data centered. Me



LAMMPS simulation for velocity 0.05 started successfully!
LAMMPS (29 Aug 2024)
Reading data file ...
  orthogonal box = (-14.674074 -12.705456 -50) to (15.015926 12.659909 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  330 atoms
  read_data CPU = 0.005 seconds
60 atoms in group C60
168 atoms in group graphene
102 atoms in group edges
Setting atom values ...
  60 settings made for type
Setting atom values ...
  102 settings made for type
Setting atom values ...
  168 settings made for type

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
- pair reaxff command: doi:10.1016/j.parco.2011.08.005
- fix qeq/reaxff command: doi:10.1016/j.parco.2011.08.005
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Neighbor list info ...
  update: every = 1 steps, delay = 0 ste



LAMMPS simulation for velocity 0.1 started successfully!
LAMMPS (29 Aug 2024)
Reading data file ...
  orthogonal box = (-14.674074 -12.705456 -50) to (15.015926 12.659909 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  330 atoms
  read_data CPU = 0.004 seconds
60 atoms in group C60
168 atoms in group graphene
102 atoms in group edges
Setting atom values ...
  60 settings made for type
Setting atom values ...
  102 settings made for type
Setting atom values ...
  168 settings made for type

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
- pair reaxff command: doi:10.1016/j.parco.2011.08.005
- fix qeq/reaxff command: doi:10.1016/j.parco.2011.08.005
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Neighbor list info ...
  update: every = 1 steps, delay = 0 step



LAMMPS simulation for velocity 0.5 started successfully!
LAMMPS (29 Aug 2024)
Reading data file ...
  orthogonal box = (-14.674074 -12.705456 -50) to (15.015926 12.659909 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  330 atoms
  read_data CPU = 0.004 seconds
60 atoms in group C60
168 atoms in group graphene
102 atoms in group edges
Setting atom values ...
  60 settings made for type
Setting atom values ...
  102 settings made for type
Setting atom values ...
  168 settings made for type

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
- pair reaxff command: doi:10.1016/j.parco.2011.08.005
- fix qeq/reaxff command: doi:10.1016/j.parco.2011.08.005
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Neighbor list info ...
  update: every = 1 steps, delay = 0 step

NameError: name 'get_dataset' is not defined

In [2]:
import subprocess
import os
from LAMMPS_gen_in_file import * 
from LAMMPS_data_preprocess import * 
from concurrent.futures import ProcessPoolExecutor

velocities = [0.02, 0.05, 0.1, 0.5]
temperature = 300

test_num = 1

def run_lammps_simulation(velocity, temperature):
    dump_dir = "./runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    dump_dir_ = "runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    
    df_graphene = get_dataset(get_files(dump_dir_, el='gr'))
    df_c60 = get_dataset(get_files(dump_dir_, el='c60'))
    
    get_plots_coords_velocities(df_graphene, name=dump_dir_ + '/graphene')
    get_plots_coords_velocities(df_c60, name=dump_dir_ + '/c60')
    
    print(f'Фуллерен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken(df_c60, alpha = 0.03)}')
    print(f'Графен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken(df_graphene, alpha = 0.03)}')

    

# Используем ProcessPoolExecutor для параллельной обработки
with ProcessPoolExecutor() as executor:
    # Запускаем задачу для каждой скорости
    futures = [executor.submit(run_lammps_simulation, velocity, temperature) for velocity in velocities]
    
    # Ожидаем завершения всех задач
    for future in futures:
        future.result()  # Это заблокирует основной поток до завершения задачи

NameError: name 'get_dataset' is not defined